# Georgian Digital Alphabet Recognition

### Preprocessing
add useful imports

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

ModuleNotFoundError: No module named 'cupy'

turns a number to a one hot array

In [2]:
def to_one_hot(values: np.ndarray, max: int) -> np.matrix:
    one_hot = np.zeros((values.size, max))
    rows = np.arange(values.size)
    one_hot[rows, values] = 1    

    return one_hot

import image data and convert images to numpy arrays

In [3]:
import matplotlib.image as im

X_arr = list[np.ndarray]()
Y_arr = list[int]()
num_chars = ord("ჰ") - ord("ა")

for i in range(num_chars):
    char = chr(ord("ა") + i)

    file_path = f"data/images/{char}"
    for image in os.listdir(file_path):
        img_data = im.imread(f"{file_path}/{image}").ravel()

        X_arr.append(np.append(img_data, np.ones(10000 - len(img_data))))
        Y_arr.append(i)

turn list of image values to numpy matrix, values to one_hot vector

In [4]:
X = np.matrix(X_arr)
Y = to_one_hot(np.asarray(Y_arr), num_chars)

X.shape, Y.shape

((28160, 10000), (28160, 32))

### Define neural network model class

In [5]:
class NeuralNetworkModel:
    def __init__(self, in_size: int, out_size: int, hidden_sizes: list[int]) -> None:
        self.layers: list[int] = [in_size] + hidden_sizes + [out_size]
        
        self.thetas = list[np.matrix]()
        self.biases = list[np.matrix]()

        for i in range(len(self.layers) - 1):
            prev_size: int = self.layers[i]
            next_size: int = self.layers[i + 1]

            self.thetas.append(np.matrix(np.random.rand(next_size, prev_size)))
            self.biases.append(np.matrix(np.random.rand(next_size, 1)))

    def __ReLU(self, X: np.matrix) -> np.matrix:
        return np.maximum(0, X)

    def __ReLU_grad(self, X: np.matrix) -> np.matrix:
        return np.where(X <= 0, 0, 1)

    def __sigmoid(self, X: np.matrix) -> np.matrix:
        # to avoid sigmoid being 0 or 1
        ephsilon = 1e-5

        sigma = np.divide(1, 1 + np.exp(-X) + ephsilon)
        sigma[sigma == 0] = ephsilon

        return sigma

    def __forward(self, input: np.matrix) -> tuple[np.matrix, list[np.matrix]]:
        result: np.matrix = input
        layer_values: list[np.matrix] = [input]

        for i in range(len(self.layers) - 1):
            # z(i)
            result = np.dot(result, self.thetas[i].T)
            result = np.add(result, self.biases[i].T)
            # a(i)
            if i != len(self.layers) - 2: 
                result = self.__ReLU(result) 
                layer_values.append(result)

            else: result = self.__sigmoid(result)

        return result, layer_values

    # forward propagation, but returns array with only 1 and 0s
    def predict(self, input: np.matrix) -> np.matrix:
        # predictions = 
        return self.__forward(input)[0]
        # return to_one_hot(np.asarray(predictions.argmax(axis=1)).ravel(), num_chars)

    def cost(self, X: np.matrix, Y: np.matrix, lambd: float) -> float:
        h, _ = self.__forward(X)

        # compute cost
        first_term = np.multiply(Y, np.log(h))
        second_term = np.multiply((1 - Y), np.log(1 - h))
        J = np.sum(first_term + second_term) / (-X.shape[0])

        # add regularization
        regularized = 0
        for thetas in self.thetas:
            regularized += np.sum(np.power(thetas, 2))

        J += np.multiply(np.divide(lambd, 2 * X.shape[0]), regularized)

        return J
    
    # backpropagation
    def gradients(self, X: np.matrix, Y: np.matrix, lambd: float) -> tuple[list[np.matrix], list[np.matrix]]:
        h, a_values = self.__forward(X)
        bias_a = [np.ones((X.shape[0], 1)) for _ in a_values]

        grads: list[np.matrix] = [np.zeros(theta.shape) for theta in self.thetas]
        biases: list[np.matrix] = [np.zeros(bias.shape) for bias in self.biases]

        last_delta: np.matrix = np.subtract(h, Y)
        grads[-1] = (grads[-1] + np.dot(last_delta.T, a_values[-1])) / X.shape[0]
        biases[-1] = (biases[-1] + np.dot(last_delta.T, bias_a[-1])) / X.shape[0]

        for i in range(len(a_values) - 1, 0, -1):
            delta = np.multiply(np.dot(last_delta, self.thetas[i]), self.__ReLU_grad(a_values[i]))

            grads[i - 1] = grads[i - 1] + np.dot(delta.T, a_values[i - 1]) / X.shape[0]
            grads[i - 1] = grads[i - 1] + np.multiply(lambd / X.shape[0], self.thetas[i - 1]) # regularization

            biases[i - 1] = (biases[i - 1] + np.dot(delta.T, bias_a[i - 1])) / X.shape[0]

            last_delta = delta

        return grads, biases

    def train(self, X: np.matrix, Y: np.matrix, alpha: float, lambd: float, max_iters: int = 1000) -> np.ndarray:
        cost = np.zeros(max_iters)

        for i in range(max_iters):
            cost[i] = self.cost(X, Y, lambd)
            theta_grads, bias_grads = self.gradients(X, Y, lambd)

            for j, _ in enumerate(self.thetas):
                self.thetas[j] = self.thetas[j] - np.multiply(alpha, theta_grads[j])
                self.biases[j] = self.biases[j] - np.multiply(alpha, bias_grads[j])

        return cost

### Build a model

In [19]:
nn = NeuralNetworkModel(10000, num_chars, [150])

In [24]:
nn.train(X, Y, 0.1, 0.1, 1)

array([1712.72377756])

In [13]:
whaat = np.asmatrix([
    im.imread(f"data/images/ა/1_aacadhn.ttf_fs_24_bc_256.ა.png").ravel(),
    im.imread(f"data/images/ც/132_aacadhn.ttf_fs_40_bc_256.ც.png").ravel(),
    np.zeros(10000)
])

In [26]:
nn.predict(whaat)

matrix([[0.65159716, 0.53017201, 0.54257803, 0.6478418 , 0.52831782,
         0.4846594 , 0.65205896, 0.53521632, 0.48016711, 0.63491729,
         0.63890211, 0.55317582, 0.53512003, 0.49922158, 0.65992325,
         0.46582678, 0.65971896, 0.57637713, 0.69448597, 0.67070361,
         0.68205264, 0.58669753, 0.63587581, 0.59020634, 0.4786046 ,
         0.60686021, 0.56713856, 0.59139268, 0.54850644, 0.51711239,
         0.52290956, 0.5036493 ],
        [0.65159716, 0.53017201, 0.54257803, 0.6478418 , 0.52831782,
         0.4846594 , 0.65205896, 0.53521632, 0.48016711, 0.63491729,
         0.63890211, 0.55317582, 0.53512003, 0.49922158, 0.65992325,
         0.46582678, 0.65971896, 0.57637713, 0.69448597, 0.67070361,
         0.68205264, 0.58669753, 0.63587581, 0.59020634, 0.4786046 ,
         0.60686021, 0.56713856, 0.59139268, 0.54850644, 0.51711239,
         0.52290956, 0.5036493 ],
        [0.65159716, 0.53017201, 0.54257803, 0.6478418 , 0.52831782,
         0.4846594 , 0.65205896, 0.